# Calculate spatial flux

This example shows how to calculate a spatial greenhouse gas (GHG) flux based on modelled input from for instance [SOMERS](https://www.nobveenweiden.nl/wp-content/uploads/2024/11/rapportage-SOMERS-2.0-technische-beschrijving.pdf). Lusos calculates a spatial flux by taking a weighed average of the median fluxes of polygons that fall within each raster cell of a 2D grid. Following the previously explained [concept](./concept.ipynb), lusos computes the area of each polygon within a raster cell. The computed areas are used as the weights to assign the computed flux to each cell. This example shows the simple steps how to calculate to spatial flux. We will use the same example area from the [coverage](./coverage.ipynb) example.

In [ ]:
import geopandas as gpd
from shapely import geometry as gmt

import lusos

# Bounding box for the example area
xmin, ymin, xmax, ymax = 111_000, 455_000, 116_000, 460_000
study_area = gpd.GeoDataFrame(geometry=[gmt.box(xmin, ymin, xmax, ymax)], crs=28992)

emissions = lusos.data.sample_emissions()

Let's first checkout the emissions data:

In [ ]:
print(emissions.head())
m = emissions.explore() # Display the emissions data on a map
study_area.explore(m=m, color="black", fill=False)

The data contains emission values for polygons in `ton/ha`. Lusos expects this unit for the input data and translates this into a flux per m2. Using a different therefore means that the output will be in the wrong unit as well and must be translated to ton/m2.

Lusos needs two columns to be present in the input data: "parcel_id" and "median". This is because lusos bases the calculation on the median emission and we need an id for each polygon to be able to identify it and correctly assign fluxes to cells. Without the presence of these columns, an error will be thrown. We can rename the column "emission_t" because this will be used as the median and make the other columns lowercase strings because "PARCEL_ID" is present in the data. 

In [ ]:
emissions.columns = emissions.columns.str.lower()
emissions.rename(columns={"emission_t": "median"}, inplace=True)
emissions.head()

Now we can define our grid again for the calculation and calculate the flux.

In [ ]:
grid = lusos.LassoGrid(xmin, ymin, xmax, ymax, 25, 25)
flux_per_m2 = lusos.calculate_somers_emissions(emissions, grid)
flux_per_m2